In [ ]:
import os
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
np.random.seed(9630)
tf.random.set_seed(9630)

print(tf.__version__)

2.7.0


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
size_input = 784
size_hidden = [128, 64]
size_output = 10
number_of_train_examples = 60000
number_of_test_examples = 10000

In [ ]:
# load datasets
mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

In [ ]:
def mnist_normalize():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

def fashion_normalize():
  (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

In [ ]:
# Split dataset into batches
(X_train, y_train), (X_test, y_test) = mnist_normalize()
# (X_train, y_train), (X_test, y_test) = fashion_normalize()

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# Define class to build MLP model
class MLP(object):
  def __init__(self, size_input, size_hidden, size_output, device = None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer (2)
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. 
    If None, the device will automatically decide during Eager Execution ('gpu')
    """

    self.size_input, self.size_hidden, self.size_output, self.device =\
    size_input, size_hidden, size_output, device

    # Initialize weights between input layer and hidden layer 1
    self.W0 = tf.Variable(tf.random.normal([self.size_input, self.size_input]))
    # Initialize biases for hidden layer 1
    self.b0 = tf.Variable(tf.random.normal([1, self.size_input]))
    # Initialize weights between input layer and hidden layer 1
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden[0]]))
    # Initialize biases for hidden layer 1
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden[0]]))
    # Initialize weights between hidden layer 1 and hidden layer 2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden[0], self.size_hidden[1]]))
    # Initialize biases for hidden layer 2
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden[1]]))
    # Initialize weights between hidden layer 2 and output layer
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden[1], self.size_output]))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    # Define variables to be updated during backpropagation
    self.variables = [self.W0, self.W1, self.W2, self.W3, self.b0, self.b1, self.b2, self.b3]



  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """

    if self.device is not None:
      with tf.device('gpu:0' if self.device == 'gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
    return self.y

  def loss(self, y_pred, y_true):
    """
    y_pred: Tensor of shape (batch_size, size_output)
    y_true: Tensor of shape (batch_size, size_output)
    """

    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

  # Dropout
  def dropout(self, X_train, dropout_rate):
    assert 0 <= dropout_rate <= 1
    
    if dropout_rate == 1: # In this case, all elements are dropped out
      return tf.zeros_like(X_train)
    elif dropout_rate == 0: # In this case, all elements are kept
      return X_train
    else:
      mask = tf.random.uniform(shape=tf.shape(X_train), minval=0, maxval=1) < 1-dropout_rate
      return tf.cast(mask, dtype=tf.float32) * X_train/(1.0-dropout_rate)

  def backward(self, X_train, y_train):
    """
    backward pass
    """

    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))

  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """

    # Cast X to float 32
    X_tf = tf.cast(X, dtype=tf.float32)
    # Remember to normalize your dataset before moving forward

    # Compute values in input layer
    what0 = tf.matmul(X_tf), self.W0) + self.b0
    hhat0 = tf.nn.relu(what0)
    dhat0 = self.dropout(hhat0, 0.2)

    # Compute values in hidden layer 1
    what1 = tf.matmul(dhat0, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)
    dhat1 = self.dropout(hhat1, 0.5)

    # Compute values in hidden layer 2
    what2 = tf.matmul(dhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)
    dhat2 = self.dropout(hhat2, 0.5)

    # Compute output
    output = tf.matmul(dhat2, self.W3) + self.b3
    output = tf.nn.softmax(output)

    # Now consider two things
    # First, look at inbuild loss functions if they work with softmax and then change this
    # Second, add tf.Softmax(output) and then return this variable
    return output

# Train Model

In [ ]:
# Set number of epochs
NUM_EPOCHS = 20

In [ ]:
def compute_acc(y_pred, y_true):
  maxid = lambda x: np.argmax(x)
  y_pred_max = np.array([maxid(i) for i in y_pred])
  y_true_max = np.array([maxid(j) for j in y_true])
  acc = sum(y_pred_max == y_true_max)/len(y_pred_max)
  return acc

In [ ]:
def compute_std(y_pred):
  y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
  std_dev = np.std(y_pred_tf)
  std_err = std_dev / np.sqrt(len(y_pred_tf))
  variance = std_dev**2
  return std_err, variance

In [ ]:
# Initialize model using GPU
# with Dropout
mlp_on_gpu = MLP(size_input, size_hidden, size_output, device = 'gpu')
time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  acc = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(9630)).batch(40)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    acc = acc + compute_acc(preds, outputs)
  print('Number of Epoch: {} - Average MSE: {}' .format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
  print('                   - Accuracy: {}' .format(acc/len(train_ds)))
 
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
# For per epoch_time
# epoch_time = Total time / Number of epochs

#Test Model

In [ ]:
test_loss_total = tf.Variable(0, dtype=tf.float32)

# test_loss_total = 0.0
test_acc = 0
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  # b = mlp_on_gpu.loss(preds, outputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
  test_acc = test_acc + compute_acc(preds, outputs)
  
# a = (test_loss_total.numpy() / X_train.shape[0])
# print(X_train.shape[0])
# print(test_loss_total.numpy())
# print(b)
print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_test.shape[0]))
print('Accuracy: {}' .format(test_acc/len(test_ds)))

# 10 Trials - Fasion MNIST

In [ ]:
# mse_fashion = []
# accuracy_fashion = []
# err_fashion = []
# var_fashion = []
# for i in range(10):
#   test_loss_total = tf.zeros([1,1], dtype=tf.float32)
#   test_acc = 0
#   test_err = 0
#   test_var = 0
#   test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
#   for inputs, outputs in test_ds:
#     preds = mlp_on_gpu.forward(inputs)
#     test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
#     mlp_on_gpu.backward(inputs, outputs)
#     test_acc = test_acc + compute_acc(preds, outputs)
#     test_err = test_err + compute_std(preds)[0]
#     test_var = test_var + compute_std(preds)[1]
#   mse_fashion.append(np.sum(test_loss_total) / X_test.shape[0])
#   accuracy_fashion.append(test_acc/len(test_ds))
#   err_fashion.append(test_err/len(test_ds))
#   var_fashion.append(test_var/len(test_ds))
#   print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
#   print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
#   print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
#   print('                   - Variance: {}' .format(test_var/len(test_ds)))
# print(mse_fashion)
# print(accuracy_fashion)
# time_taken = time.time() - time_start

# print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

In [1]:
# print(np.mean(accuracy_fashion))
# print(np.var(accuracy_fashion))
# plt.plot(accuracy_fashion, label = 'mean:0.76, var:0.00002')
# plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
# plt.xlabel('Accuracy')
# plt.ylabel('Count')
# plt.title('Dropout - Fashion MNIST')
# plt.show()

# 10 Trials - MNIST

In [3]:
mse_mnist = []
accuracy_mnist = []
err_mnist = []
var_mnist = []
for i in range(10):
  test_loss_total = tf.zeros([1,1], dtype=tf.float32)
  test_acc = 0
  test_err = 0
  test_var = 0
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    test_acc = test_acc + compute_acc(preds, outputs)
    test_err = test_err + compute_std(preds)[0]
    test_var = test_var + compute_std(preds)[1]
  mse_mnist.append(np.sum(test_loss_total) / X_test.shape[0])
  accuracy_mnist.append(test_acc/len(test_ds))
  err_mnist.append(test_err / len(test_ds))
  var_mnist.append(test_var/len(test_ds))
  print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
  print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
  print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
  print('                   - Variance: {}' .format(test_var/len(test_ds)))
 
print(mse_mnist)
print(accuracy_mnist)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

In [4]:
print(np.mean(accuracy_mnist))
print(np.var(accuracy_mnist))
plt.plot(accuracy_mnist, label = 'mean:0.8011, var:0.000027')
plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
plt.xlabel('Accuracy')
plt.ylabel('Count')
plt.title('Dropout - MNIST')
plt.show()